In [1]:
# Requirements 
# !pip3 install pandas numpy seaborn 
import os
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
!bash RQ6_Nov19.sh
!bash RQ6_Oct19.sh

2019-Nov.csv
2019-Oct.csv


In [3]:
df_labels = ['2019-Oct.csv.rq6', '2019-Nov.csv.rq6']

In [4]:
def get_file_path(df_label='2019-Oct', aws=False):
    if aws:
        return f"s3://Users/tommy/Desktop/codici/datasets/{df_label}.csv"
    return f"datasets/{df_label}.csv"

In [5]:
nov_rq6 = pd.read_csv('/Users/tommy/Desktop/codici/datasets/2019-Nov.csv.rq6.csv')

In [6]:
octo_rq6 = pd.read_csv('/Users/tommy/Desktop/codici/datasets/2019-Oct.csv.rq6.csv')

In [7]:
tot_rq6 = pd.concat([octo_rq6,nov_rq6]) #about a minute

In [8]:
tot_rq6.head()

,event_type,product_id,category_code,user_id
0,view,44600062,NaN,541312140
1,view,3900821,appliances.environment.water_heater,554748717
2,view,17200506,furniture.living_room.sofa,519107250
3,view,1307067,computers.notebook,550050854
4,view,1004237,electronics.smartphone,535871217


In [9]:
subRQ6 = tot_rq6

In [10]:
subRQ6.head()

,event_type,product_id,category_code,user_id
0,view,44600062,NaN,541312140
1,view,3900821,appliances.environment.water_heater,554748717
2,view,17200506,furniture.living_room.sofa,519107250
3,view,1307067,computers.notebook,550050854
4,view,1004237,electronics.smartphone,535871217


In [11]:
# obtaining the number of product sold:
# by taking only rows about purchasing
# and at the end we count them (.shape[0])
# shape[0] are the rows of the dataset where the event is equal to purchase
# so it rapresents the product sold

df_prod_sold =  subRQ6[subRQ6['event_type']=='purchase']
prod_sold = df_prod_sold.shape[0]

In [12]:
prod_sold

1659788

In [13]:
# in a similar way we calculate the number of user:
# this time we group by 'user_id' so we obtain the exact users's number
# we extract that number by the command shape[0] (number of rows)
users = subRQ6.groupby('user_id').product_id.count().shape[0]

In [14]:
users

5316649

In [15]:
#finally we calculate the actual purchase rate value
purchase_rate = prod_sold/users
round(purchase_rate,4)

0.3122

In [16]:
#same think of before
df_views = subRQ6[subRQ6['event_type']=='view']
views = df_views.shape[0]

In [17]:
#conversion rate calculated by following the previous formula
conversion_rate = purchase_rate / views
conversion_rate

2.992144008055359e-09

In [18]:
#dropping nas values for both dataframe
df_views = df_views.dropna()
df_prod_sold = df_prod_sold.dropna()


In [19]:
#purchase rate of each category
df_prod_sold['category_code'] = [el.split('.')[0] for el in list(df_prod_sold['category_code']) if type(el) == str]

In [20]:
number_of_sales = df_prod_sold.groupby('category_code').product_id.count()
number_of_sales

category_code
accessories       3741
apparel          22217
appliances      174022
auto             21339
computers        62332
construction     16500
country_yard       132
electronics     916667
furniture        19843
kids             11648
medicine           654
sport             2725
stationery         325
Name: product_id, dtype: int64

In [21]:
# obtaining number of users for each categories
# first step: grouping for categories subRQ6
grouped_subRQ6 = subRQ6.groupby("category_code")

In [22]:
#now we want to extract the number of unique users for each category
user_of_each_cat = grouped_subRQ6.agg({"user_id": "nunique"})
user_of_each_cat.iloc[:,0]

category_code
accessories.bag         102249
accessories.umbrella      3853
accessories.wallet       19867
apparel.belt              2941
apparel.costume         145585
                         ...  
sport.ski                 4769
sport.snowboard           6137
sport.tennis              3370
sport.trainer            24029
stationery.cartrige       5690
Name: user_id, Length: 129, dtype: int64

In [23]:
#converting into a series
series_of_unique_users = user_of_each_cat.iloc[:,0]

In [24]:
#calculating the purchase rate => 
# number of product sold for each catogory / number of unique user for each category
purchase_rate_for_cat = number_of_sales.divide(series_of_unique_users)
round(purchase_rate_for_cat,4).sort_values(ascending=False)


category_code
accessories            NaN
accessories.bag        NaN
accessories.umbrella   NaN
accessories.wallet     NaN
apparel                NaN
                        ..
sport.snowboard        NaN
sport.tennis           NaN
sport.trainer          NaN
stationery             NaN
stationery.cartrige    NaN
Length: 142, dtype: float64

In [25]:
# now for the last step of this request 
# we calculate the conversion rate of each categories
# we have yet the purchase rate of product for each category
# in a similar way we are going to calculate the numer of views
# for each category and then we simply divide them.

df_views['category_code'] = [el.split('.')[0] for el in list(df_views['category_code']) if type(el) == str]


In [26]:
number_of_views = df_views.groupby('category_code').product_id.count()
number_of_views


category_code
accessories       616475
apparel          4477518
appliances      12837916
auto             2157706
computers        6297977
construction     1759762
country_yard       31255
electronics     37026582
furniture        3295250
kids             1292002
medicine           34738
sport             415087
stationery         19323
Name: product_id, dtype: int64

In [27]:
coversion_rate_for_categories = purchase_rate_for_cat.divide(number_of_views)
coversion_rate_for_categories.sort_values(ascending=False)


category_code
accessories            NaN
accessories.bag        NaN
accessories.umbrella   NaN
accessories.wallet     NaN
apparel                NaN
                        ..
sport.snowboard        NaN
sport.tennis           NaN
sport.trainer          NaN
stationery             NaN
stationery.cartrige    NaN
Length: 142, dtype: float64

In [28]:
#now we apply a traslation to plot better the results
coversion_for_plot = ((coversion_rate_for_categories)*10**8).sort_values(ascending=False)
